In [1]:
import gurobi as gb
from gurobipy import GRB

import networkx as nx
import numpy as np
import math

In [2]:
from functions_library import *

In [3]:
opt_mod = gb.Model(name="linear programm")
#opt_mod.setParam("IterationLimit",400000)
#opt_mod.setParam("DegenMoves",0);

opt_mod.setParam("MIPFocus",2);



Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-02
Set parameter MIPFocus to value 2


In [4]:
k = 4
G = init_ring(2**k)
D = init_uniformDemand_matrix_symmetric(G)
nodesPairListNoDuplication = complete_node_pair_list_noDuplication(G)
n = 2**k

In [5]:
e = opt_mod.addMVar((n,n), name="e", vtype="I",lb=0,ub=1)
SP = opt_mod.addMVar((n,n,n), name="sp", vtype="I", lb=0, ub=(n/2)+1)
helper = opt_mod.addMVar((n,n,n), name="helper", vtype="I", lb=0, ub=n+2)
opt_mod.update()

In [6]:
# Adjacency Matrix (e) Constraints
opt_mod.addConstrs((e[i,(i+1) % n] == 1 for i in range(n)), name="c-ring")
opt_mod.addConstrs((e[i,i] == 0 for i in range(n)), name="c-noSelfEdge")
opt_mod.addConstrs((e[i,j] == e[j,i] for i in range(n) for j in range(n)), name="c-undirected")

maxNumberE =  k+1 #math.log(n,2)-1 + 2
opt_mod.addConstrs((e[i,:].sum() <= maxNumberE for i in range(n)), name="c-logE")
opt_mod.update() 


opt_mod.addConstrs((SP[0,i,j] == (1-e[i][j])*((n/2)+1)+(e[i][j]) for i in range(n) for j in range(n)), name="c-DP_init")
 
# Ensuring that x shows a correct path
for k in range(1,n):
    for i in range(n):
        for j in range(n):
            if i!=j:
                opt_mod.addConstr(helper[k-1,i,j] == gb.quicksum([SP[k-1,i,k],SP[k-1,k,j]]), name="c-DP-helper")
                opt_mod.addConstr(SP[k,i,j] == gb.min_(SP[k-1,i,j],helper[k-1,i,j]), name="c-DP0")
                #pt_mod.addConstr((SP[k-1,i,k] + SP[k-1,k,j]) + n*helper[k-1,i,j] <= SP[k-1,k,j])
                #opt_mod.addConstr(SP[k,i,j] == ((1-helper[k-1,i,j])*(SP[k-1,i,k] + SP[k-1,k,j])) + ((helper[k-1,i,j])*(SP[k-1,i,j])))
                #opt_mod.addConstr(SP[k,i,j] <= SP[k-1,i,k] + SP[k-1,k,j])
                #opt_mod.addConstr((SP[k-1,i,j] >= SP[k-1,i,k] + SP[k-1,k,j]) >> (SP[k,i,j]==SP[k-1,i,k] + SP[k-1,k,j]))

In [ ]:
#print(len(x[0,1,x,y]))
opt_mod.setObjective(gb.quicksum([SP[n-1,i,j]*D[i][j] for (i,j) in nodesPairListNoDuplication]), GRB.MINIMIZE)
opt_mod.update()


# Run
#opt_mod.display()
opt_mod.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4160 rows, 8448 columns and 12080 nonzeros
Model fingerprint: 0x7182fbca
Model has 3600 general constraints
Variable types: 0 continuous, 8448 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [7e-02, 7e-02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 9e+00]
Presolve added 6246 rows and 2060 columns
Presolve time: 0.82s
Presolved: 10406 rows, 10508 columns, 28820 nonzeros
Variable types: 0 continuous, 10508 integer (2946 binary)
Root relaxation presolve removed 4020 rows and 4020 columns
Root relaxation presolved: 6386 rows, 6488 columns, 22210 nonzeros


Root relaxation: objective 8.000000e+00, 1593 iterations, 0.07 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

  3567  1429    8.80159   27 4276   13.60000    8.06667  40.7%   607  536s
  3586  1433    8.46713   27 4434   13.60000    8.06667  40.7%   611  544s
  3611  1438    8.46714   28 4386   13.60000    8.06667  40.7%   620  550s
  3623  1450    8.46724   31 4629   13.60000    8.06667  40.7%   630  555s
  3635  1455    8.53390   32 4646   13.60000    8.06667  40.7%   636  572s
  3643  1468    8.49495   32 4308   13.60000    8.06667  40.7%   641  581s
  3656  1479    8.54452   33 4490   13.60000    8.06667  40.7%   652  589s
  3667  1495    8.60128   32 4221   13.60000    8.06667  40.7%   662  595s
  3685  1506    8.68940   33 3999   13.60000    8.06667  40.7%   672  603s
  3720  1519    8.66712   33 4049   13.60000    8.06667  40.7%   677  610s
  3733  1534    8.66715   34 4060   13.60000    8.06667  40.7%   683  618s
  3748  1546    8.73381   35 4092   13.60000    8.06667  40.7%   691  626s
  3760  1559    8.80267   37 4201   13.60000    8.06667  40.7%   697  636s
  3783  1571    9.49163  

In [ ]:
G_sol = nx.Graph()
for i in range(len(G.nodes)): 
    G_sol.add_node(i)

for i in G_sol.nodes:
    for j in range(i+1,len(G_sol.nodes)):
        if e[i,j].X == 1.0:
            G_sol.add_edge(i,j)
pos = nx.circular_layout(G_sol)
nx.draw_networkx(G_sol,pos=pos,with_labels=True)
print(calc_cost(G_sol,D))

In [ ]:
print(G_sol.edges)